In [1]:
import xml.dom.minidom
import xml.etree.ElementTree as ET
from xml.dom.minidom import Node
import pandas as pd

In [37]:
Historical_DOU_PtDays = pd.read_csv("CY2020 DOU PtDays Data.csv")
Q1 = Historical_DOU_PtDays[Historical_DOU_PtDays.Month.isin([1,2,3])]
Q2 = Historical_DOU_PtDays[Historical_DOU_PtDays.Month.isin([4,5,6])]
Q3 = Historical_DOU_PtDays[Historical_DOU_PtDays.Month.isin([7,8,9])]
Q4 = Historical_DOU_PtDays[Historical_DOU_PtDays.Month.isin([10,11,12])]

In [19]:
Pt_Days_Data = pd.read_csv("Q2_2021 Clean PtDays Data.csv")

In [20]:
set(Pt_Days_Data.Hospital)

{'EDH', 'EHH', 'EJCH', 'ELTAC', 'ESJH', 'EUH', 'EUHM', 'EUOSH'}

In [21]:
# The variables needed for this function are the: Hospital abbreviation, Hospital NDNQI template, Hospital NDNQI
# code, Calendar year being submitted, and Calendar quarter being submitted
def NDNQI_PtDays_XML(Selected_Hosp,Hosp_XML_Template,Hosp_NDNQI_Code,Calendar_Year,Calendar_Quarter):
    
    # The first step takes only the data for this selected hospital
    Hosp_data = Pt_Days_Data[Pt_Days_Data.Hospital == Selected_Hosp]
    # Now this is a List of this Hospital's Units
    Hosp_Units = list(set(Hosp_data.UnitName))

    #This is the current XML template for this hospital and their NDNQI units for this metric
    tree = xml.dom.minidom.parse(Hosp_XML_Template)
    # This is the xml's entire UnitYrQtr node
    unit_nodes = tree.getElementsByTagName("UnitYrQtr")

    # Make sure the correct NDNQI code, year, and quarter are listed
    H = tree.getElementsByTagName("HospitalCode")[0].childNodes[0].nodeValue = Hosp_NDNQI_Code
    Y = tree.getElementsByTagName("Year")[0].childNodes[0].nodeValue = Calendar_Year
    Q = tree.getElementsByTagName("Quarter")[0].childNodes[0].nodeValue = Calendar_Quarter
    

    # This loops through each separate UnitName from the Template XML file and replaces it the correct NDNQI
    # name for the units. 
    for idx, unit in enumerate(unit_nodes):
        unit.getElementsByTagName('UnitName')[0].childNodes[0].nodeValue = Hosp_Units[idx]

        # Now grab each of the 3 months for that unit from the Hospital data
        new_data = Hosp_data[Hosp_data.UnitName == Hosp_Units[idx]]
        # Sort the data by month
        new_data = new_data.sort_values('Month')

        # Take each xml Month and replace it with the months from the new_data for that unit
        month_nodes = unit.getElementsByTagName("UnitMonth")
        for idx2, month in enumerate(month_nodes):
            month.getElementsByTagName('Month')[0].childNodes[0].nodeValue = new_data.Month.iloc[idx2]

            # This is looping through the 'xPatientDaysMethodFID', 'PtDaysMNCensus' columns from the data
            for column in new_data.columns[3:]:

                #Lastly it is replacing the values in the 'XML Template' with the correct values from the data.
                new_value = new_data[column].iloc[idx2]
                month.getElementsByTagName(column)[0].childNodes[0].nodeValue = new_value



    # Now, we need to write and save this XML file as its own NDNQI Quarter and Year and Hospital to upload
    # into NDNQI.
    tree.writexml(open('Q{}_{} {} PtDays.xml'.format(Calendar_Quarter, Calendar_Year,Selected_Hosp), 'w'))

## EUOSH

In [5]:
EUOSH_data = Pt_Days_Data[Pt_Days_Data.Hospital == 'EUOSH']
EUOSH_data

,Hospital,UnitName,Month,xPatientDaysMethodFID,PtDaysMNCensus
29,EUOSH,OSH2,4,Method 5 - Patient Days from Multiple Census R...,307.36
46,EUOSH,OSH4 / OSH3,4,Method 5 - Patient Days from Multiple Census R...,394.66
56,EUOSH,OSH2,6,Method 5 - Patient Days from Multiple Census R...,275.64
61,EUOSH,OSH ICU,6,Method 5 - Patient Days from Multiple Census R...,60.00
65,EUOSH,OSH4 / OSH3,5,Method 5 - Patient Days from Multiple Census R...,423.01
83,EUOSH,OSH ICU,5,Method 5 - Patient Days from Multiple Census R...,85.54
176,EUOSH,OSH4 / OSH3,6,Method 5 - Patient Days from Multiple Census R...,459.96
196,EUOSH,OSH2,5,Method 5 - Patient Days from Multiple Census R...,225.58
219,EUOSH,OSH ICU,4,Method 5 - Patient Days from Multiple Census R...,58.25


In [6]:
NDNQI_PtDays_XML('EUOSH','2020 EUOSH PtDays Template.xml','SBI556',2021,2)

## EUH

In [7]:
EUH_data = Pt_Days_Data[Pt_Days_Data.Hospital == 'EUH']
EUH_data.head()

,Hospital,UnitName,Month,xPatientDaysMethodFID,PtDaysMNCensus
5,EUH,6T Trans,4,Method 5 - Patient Days from Multiple Census R...,1072.97
6,EUH,5T N ICU,6,Method 5 - Patient Days from Multiple Census R...,459.04
15,EUH,8E Step Down,6,Method 5 - Patient Days from Multiple Census R...,695.88
17,EUH,9T BMT,6,Method 5 - Patient Days from Multiple Census R...,1264.50
23,EUH,9T BMT,4,Method 5 - Patient Days from Multiple Census R...,1162.81


In [8]:
NDNQI_PtDays_XML('EUH','2020 EUH PtDays Template.xml','JXE432',2021,2)

## EUHM

In [9]:
EUHM_data = Pt_Days_Data[Pt_Days_Data.Hospital == 'EUHM']
EUHM_data.head()

,Hospital,UnitName,Month,xPatientDaysMethodFID,PtDaysMNCensus
2,EUHM,21 ICU,5,Method 5 - Patient Days from Multiple Census R...,378.00
3,EUHM,U21S Gen Surg,4,Method 5 - Patient Days from Multiple Census R...,981.57
4,EUHM,SCN,4,Method 5 - Patient Days from Multiple Census R...,1249.00
7,EUHM,U52 Renal,5,Method 5 - Patient Days from Multiple Census R...,695.07
16,EUHM,21 ICU,6,Method 5 - Patient Days from Multiple Census R...,310.46


In [10]:
NDNQI_PtDays_XML('EUHM','2020 EUHM PtDays Template.xml','PVF155',2021,2)

## EJCH

In [11]:
EJCH_data = Pt_Days_Data[Pt_Days_Data.Hospital == 'EJCH']
EJCH_data.head()

,Hospital,UnitName,Month,xPatientDaysMethodFID,PtDaysMNCensus
0,EJCH,JC3,4,Method 5 - Patient Days from Multiple Census R...,1046.18
1,EJCH,JC Oncology,4,Method 5 - Patient Days from Multiple Census R...,367.78
41,EJCH,JC Oncology,5,Method 5 - Patient Days from Multiple Census R...,403.66
51,EJCH,JC4,4,Method 5 - Patient Days from Multiple Census R...,997.01
71,EJCH,JC4,5,Method 5 - Patient Days from Multiple Census R...,1028.61


In [12]:
NDNQI_PtDays_XML('EJCH','2020 EJCH PtDays Template.xml','YTD069',2021,2)

## EJSH

In [13]:
ESJH_data = Pt_Days_Data[Pt_Days_Data.Hospital == 'ESJH']
ESJH_data.head()

,Hospital,UnitName,Month,xPatientDaysMethodFID,PtDaysMNCensus
9,ESJH,4 West Telemetry (Renal-Urology),4,Method 5 - Patient Days from Multiple Census R...,600.05
10,ESJH,4 East Telemetry (Neurovascular),4,Method 5 - Patient Days from Multiple Census R...,597.64
12,ESJH,5 East Internal Medicine,6,Method 5 - Patient Days from Multiple Census R...,550.07
13,ESJH,3 Southwest GI,4,Method 5 - Patient Days from Multiple Census R...,458.03
14,ESJH,3 East Telemetry (CV Step- down),4,Method 5 - Patient Days from Multiple Census R...,577.97


In [14]:
NDNQI_PtDays_XML('ESJH','2020 ESJH PtDays Template.xml','XAR317',2021,2)

## EDH

In [15]:
EDH_data = Pt_Days_Data[Pt_Days_Data.Hospital == 'EDH']
EDH_data.head()

,Hospital,UnitName,Month,xPatientDaysMethodFID,PtDaysMNCensus
8,EDH,EDH 44,5,Method 5 - Patient Days from Multiple Census R...,674.54
11,EDH,EDH 26 PCU,4,Method 5 - Patient Days from Multiple Census R...,341.33
30,EDH,EDH 32/33,4,Method 5 - Patient Days from Multiple Census R...,637.50
36,EDH,EDH 36,5,Method 5 - Patient Days from Multiple Census R...,986.00
37,EDH,EDH 36,6,Method 5 - Patient Days from Multiple Census R...,966.00


In [16]:
NDNQI_PtDays_XML('EDH','2021 EDH PtDays Template.xml','NMW346',2021,2)

## EHH

In [22]:
EHH_data = Pt_Days_Data[Pt_Days_Data.Hospital == 'EHH']
EHH_data

,Hospital,UnitName,Month,xPatientDaysMethodFID,PtDaysMNCensus
18,EHH,EHH 4 ICU,5,Method 5 - Patient Days from Multiple Census R...,144.00
57,EHH,EHH 4,5,Method 5 - Patient Days from Multiple Census R...,730.29
64,EHH,EHH 5,5,Method 5 - Patient Days from Multiple Census R...,792.63
73,EHH,EHH 5,4,Method 5 - Patient Days from Multiple Census R...,350.21
114,EHH,EHH 5,6,Method 5 - Patient Days from Multiple Census R...,959.13
169,EHH,EHH 4,6,Method 5 - Patient Days from Multiple Census R...,756.13
178,EHH,EHH 4 ICU,6,Method 5 - Patient Days from Multiple Census R...,189.00
199,EHH,EHH 4 ICU,4,Method 5 - Patient Days from Multiple Census R...,189.00
201,EHH,EHH 4,4,Method 5 - Patient Days from Multiple Census R...,693.42


In [23]:
NDNQI_PtDays_XML('EHH','2021 EHH PtDays Template.xml','WXX116',2021,2)

## ELTAC

In [24]:
ELTAC_data = Pt_Days_Data[Pt_Days_Data.Hospital == 'ELTAC']
ELTAC_data

,Hospital,UnitName,Month,xPatientDaysMethodFID,PtDaysMNCensus
70,ELTAC,ELTAC 5 ICU,6,Method 5 - Patient Days from Multiple Census R...,156.0
110,ELTAC,ELTAC 4/5,6,Method 5 - Patient Days from Multiple Census R...,1084.0
112,ELTAC,ELTAC 5 ICU,4,Method 5 - Patient Days from Multiple Census R...,88.0
172,ELTAC,ELTAC 4/5,5,Method 5 - Patient Days from Multiple Census R...,1141.0
212,ELTAC,ELTAC 4/5,4,Method 5 - Patient Days from Multiple Census R...,1232.0
223,ELTAC,ELTAC 5 ICU,5,Method 5 - Patient Days from Multiple Census R...,115.0


In [25]:
NDNQI_PtDays_XML('ELTAC','2021 ELTAC PtDays Template.xml','XLS464',2021,2)